In [ ]:
!pip install catboost

     |████████████████████████████████| 76.3 MB 21 kB/s 


In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/pre_P/train_data.csv')
df_test = pd.read_csv('/content/drive/MyDrive/pre_P/test_data.csv')

In [ ]:
cat_cols = ['Gender', 'City_Category','Customer_Category']
cols_to_drop = ['Customer_ID','is_train']

In [ ]:
B1_cols = [col for col in list(df_train.columns) if col.startswith('B1')]

In [ ]:
B2_cols = [col for col in list(df_train.columns) if col.startswith('B2')]

In [ ]:
cat_cols = B1_cols + cat_cols
target_cols = B2_cols

In [ ]:
df_train = df_train.drop(columns = cols_to_drop)

df_test = df_test.drop(columns = cols_to_drop)

In [ ]:
#########################################
######################################
df_train['Gender'] = df_train['Gender'].map({'Male':1, 'Female':0})
df_test['Gender'] = df_test['Gender'].map({'Male':1, 'Female':0})
#Train
ordinal= df_train['City_Category'].value_counts().index
ordinal = {k:i for i,k in enumerate(ordinal,1)}
df_train['City_Category'] = df_train['City_Category'].map(ordinal)

ordinal= df_train['Customer_Category'].value_counts().index
ordinal = {k:i for i,k in enumerate(ordinal,1)}
df_train['Customer_Category'] = df_train['Customer_Category'].map(ordinal)

#Test
ordinal= df_test['City_Category'].value_counts().index
ordinal = {k:i for i,k in enumerate(ordinal,1)}
df_test['City_Category'] = df_test['City_Category'].map(ordinal)

ordinal= df_test['Customer_Category'].value_counts().index
ordinal = {k:i for i,k in enumerate(ordinal,1)}
df_test['Customer_Category'] = df_test['Customer_Category'].map(ordinal)


In [ ]:
#transformation

df_train['Age'] = np.log10(df_train['Age'])
df_train['Vintage'] = np.log10(df_train['Vintage'])
df_train['current_holdings'] = np.log10(df_train['current_holdings'])

df_test['Age'] = np.log10(df_test['Age'])
df_test['Vintage'] = np.log10(df_test['Vintage'])
df_test['current_holdings'] = np.log10(df_test['current_holdings'])

In [ ]:
df_train.head()

,Gender,Age,Vintage,Is_Active,City_Category,Customer_Category,B1_P16,B1_P13,B1_P20,B1_P11,B1_P8,B1_P17,B1_P21,B1_P12,B1_P10,B1_P19,B1_P2,B1_P00,B1_P18,B1_P15,B1_P6,B1_P9,B1_P7,B1_P3,B1_P5,B1_P4,B1_P1,B1_P14,B2_P8,B2_P3,B2_P00,B2_P6,B2_P12,B2_P16,B2_P1,B2_P9,B2_P10,B2_P13,B2_P4,B2_P5,B2_P7,B2_P11,B2_P2,B2_P15,B2_P17,B2_P14,B2_P20,B2_P18,current_holdings
0,1,1.612784,1.146128,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000
1,0,1.672098,1.146128,1,1,2,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.301030
2,0,1.770852,1.146128,0,2,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000
3,0,1.505150,1.204120,0,1,2,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.301030
4,1,1.477121,1.176091,0,2,1,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.477121


In [ ]:
from sklearn.model_selection import train_test_split

X_train = df_train.drop(columns = target_cols)
#######################################################
df_test = df_test.drop(columns = target_cols)
y_train = df_train[target_cols]

In [ ]:
for cols in y_train.columns:
    print(cols)
    print(y_train[cols].unique())

B2_P8
[1 0]
B2_P3
[0 1]
B2_P00
[0 1]
B2_P6
[0 1]
B2_P12
[0 1]
B2_P16
[0 1]
B2_P1
[0 1]
B2_P9
[0 1]
B2_P10
[0 1]
B2_P13
[0 1]
B2_P4
[0 1]
B2_P5
[0 1]
B2_P7
[0 1]
B2_P11
[0 1]
B2_P2
[0 1]
B2_P15
[0 1]
B2_P17
[0 1]
B2_P14
[0 1]
B2_P20
[0 1]
B2_P18
[0 1]


In [ ]:
#Stacking Regressor
from sklearn.multioutput import MultiOutputClassifier
import lightgbm
from sklearn.ensemble import StackingClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
import xgboost
r1 = CatBoostClassifier()
r3 = xgboost.XGBClassifier()
r4 = GradientBoostingClassifier()
r5 = RandomForestClassifier()
r6 = lightgbm.LGBMClassifier()

#er = StackingRegressor([('cat', r1), ('ada', r2), ('xg', r3), ('gb', r4), ('rf', r5)])
classifier = MultiOutputClassifier(xgboost.XGBClassifier())

In [ ]:
classifier.fit(X_train, y_train)

MultiOutputClassifier(estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                              colsample_bylevel=1,
                                              colsample_bynode=1,
                                              colsample_bytree=1, gamma=0,
                                              learning_rate=0.1,
                                              max_delta_step=0, max_depth=3,
                                              min_child_weight=1, missing=None,
                                              n_estimators=100, n_jobs=1,
                                              nthread=None,
                                              objective='binary:logistic',
                                              random_state=0, reg_alpha=0,
                                              reg_lambda=1, scale_pos_weight=1,
                                              seed=None, silent=None,
                                              subsample=

In [ ]:
X_train.shape

(37748, 29)

In [ ]:
df_test.shape

(20327, 29)

In [ ]:
preds = classifier.predict_proba(df_test.values)

In [ ]:
preds_cal = []
for item in preds:
    preds_cal.append( [local_item[1] for local_item in item] )

In [ ]:
import numpy as np

df_catboost = pd.DataFrame(np.array(preds_cal).T, columns=B2_cols)
df_catboost.head()

,B2_P8,B2_P3,B2_P00,B2_P6,B2_P12,B2_P16,B2_P1,B2_P9,B2_P10,B2_P13,B2_P4,B2_P5,B2_P7,B2_P11,B2_P2,B2_P15,B2_P17,B2_P14,B2_P20,B2_P18
0,0.506997,0.100786,0.001222,0.036805,0.004548,0.000045,0.059604,0.051562,0.242848,0.000111,0.176556,0.037765,0.028066,0.000062,0.000053,0.000075,0.000048,0.000053,0.000050,0.000039
1,0.462487,0.092874,0.001320,0.032776,0.003501,0.000041,0.345427,0.034684,0.185787,0.000111,0.114804,0.029007,0.030292,0.000062,0.000053,0.000086,0.000045,0.000071,0.000048,0.000039
2,0.064248,0.006049,0.146633,0.029497,0.012451,0.614646,0.005321,0.008383,0.023331,0.165840,0.009148,0.004219,0.009309,0.002574,0.003400,0.001200,0.001992,0.000079,0.000044,0.000068
3,0.292307,0.040343,0.000289,0.052982,0.107439,0.004170,0.034324,0.091355,0.111811,0.000761,0.072507,0.040305,0.017552,0.000298,0.000097,0.000083,0.000058,0.000100,0.000102,0.000093
4,0.195229,0.051464,0.556012,0.029498,0.079235,0.000116,0.141616,0.020589,0.009595,0.000748,0.009148,0.003524,0.016599,0.000060,0.000049,0.000083,0.000047,0.000078,0.000050,0.000039


In [ ]:
df_catboost.to_csv('catboost_preds.csv', index=False)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/AV_NOV/test.csv')
df_cat = pd.read_csv('/content/catboost_preds.csv')

In [ ]:
df_cat.shape

(20327, 20)

In [ ]:
df = pd.concat([df, df_cat], axis=1)

In [ ]:
total_cols = [col for col in df.columns if col.startswith('B2_')]

In [ ]:
def get_labels(row):
    labels = []
    max_val = -1
    max_label = ''
    label_dict = {}
    for col in total_cols:
        if row[col] > max_val:
            max_val = row[col]
            max_label = col.split('_')[-1] 

        if row[col] > 0:
            labels.append(col.split('_')[-1])
            label_dict[col.split('_')[-1]] = row[col]
    
    if len(labels) == 0:
        labels.append(max_label)
        
    if len(labels) > 3:
        sorted_dict = dict(sorted(label_dict.items(), key=lambda item: item[1], reverse=True))
        print(sorted_dict)
        labels = []
        labels.extend(list(sorted_dict.keys())[0:3])
        
    return str(labels)

In [ ]:
df['Product_Holding_B2'] = df.apply( get_labels, axis=1 )

Streaming output truncated to the last 5000 lines.
{'P16': 0.3667552, 'P13': 0.21247482, 'P00': 0.13987832, 'P8': 0.10518486, 'P12': 0.06319437, 'P6': 0.029497435, 'P2': 0.023237444, 'P10': 0.02124924, 'P9': 0.0102893505, 'P11': 0.009267926, 'P7': 0.008463868000000001, 'P3': 0.0076170815, 'P4': 0.005726726999999999, 'P1': 0.004594313, 'P5': 0.0025176408, 'P17': 0.0014677506, 'P15': 0.0008245323599999999, 'P14': 8.513321e-05, 'P18': 6.759998e-05, 'P20': 4.412708e-05}
{'P8': 0.31412557, 'P00': 0.17063168, 'P6': 0.1404723, 'P12': 0.0892946, 'P3': 0.068049036, 'P9': 0.045645595, 'P10': 0.021957904, 'P7': 0.018360624, 'P1': 0.014523318, 'P4': 0.013769122, 'P5': 0.00878781, 'P16': 0.0021037974, 'P13': 0.0006995514999999999, 'P2': 0.0006012993, 'P11': 0.00034410326, 'P15': 0.000119000775, 'P20': 9.8957775e-05, 'P14': 5.6860503e-05, 'P17': 4.4536562000000006e-05, 'P18': 3.9083922999999996e-05}
{'P8': 0.7197838000000001, 'P12': 0.564339, 'P6': 0.12509891, 'P9': 0.11957546, 'P3': 0.065802045, 'P

In [ ]:
df[['Customer_ID', 'Product_Holding_B2']].to_csv('submissionstack0.csv', index=False)

In [ ]:
fr= pd.read_csv('/content/submissionstack0.csv')
fr.head()

,Customer_ID,Product_Holding_B2
0,CC372708,"['P8', 'P10', 'P4']"
1,CC216072,"['P8', 'P1', 'P10']"
2,CC387629,"['P16', 'P13', 'P00']"
3,CC389228,"['P8', 'P10', 'P12']"
4,CC394445,"['P00', 'P8', 'P1']"
